![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# Strategy 1: Buy and hold

Buy and hold generates profit from the risk premia of market risk exposition.

A buy and hold strategy of 1 instrument is the most simple strategy. It requires several considerations that will be touched on in this notebook

## Data aquisition

In [ ]:
from datetime import datetime, timedelta
from QuantConnect import Resolution
from QuantConnect.Securities import Futures

# Initialize QuantBook
qb = QuantBook()

# Add Future Subscriptions for ES and MES with Daily resolution
es_future = qb.AddFuture(Futures.Indices.SP500EMini, Resolution.Daily)
mes_future = qb.AddFuture(Futures.Indices.MicroSP500EMini, Resolution.Daily)

# Set Historical Data Range
start_date = datetime(2007, 1, 1)
end_date = datetime.now()

# Request Historical Data for ES and MES futures
es_history = qb.History([es_future.Symbol], start_date, end_date, Resolution.Daily)
mes_history = qb.History([mes_future.Symbol], start_date, end_date, Resolution.Daily)


# Save data to Object Store (optional)
#qb.ObjectStore.Save("ES_Future_Data", es_df.to_csv(index=False))
#qb.ObjectStore.Save("MES_Future_Data", mes_df.to_csv(index=False))

es_history.reset_index(level = 0, drop = True, inplace=True)
mes_history.reset_index(level = 0, drop = True, inplace=True)
# Display the first few rows of the historical data
print("ES Future Data:")
print(es_history.head())

print("\nMES Future Data:")
print(mes_history.head())



ES Future Data:
                               askclose      askhigh       asklow  \
symbol time                                                         
ES 1S1 2007-01-03 19:00:00  1380.443221  1394.484909  1371.969788   
       2007-01-04 19:00:00  1383.348398  1386.253575  1372.211887   
       2007-01-05 19:00:00  1371.727690  1383.348398  1369.790906   
       2007-01-08 19:00:00  1378.022240  1379.474829  1368.580415   
       2007-01-09 19:00:00  1376.327554  1379.959025  1369.548808   

                                askopen  asksize     bidclose      bidhigh  \
symbol time                                                                  
ES 1S1 2007-01-03 19:00:00  1388.674556    173.0  1380.201123  1394.242811   
       2007-01-04 19:00:00  1380.443221    510.0  1383.106300  1386.011477   
       2007-01-05 19:00:00  1383.348398     30.0  1371.243494  1383.106300   
       2007-01-08 19:00:00  1371.727690    103.0  1377.780142  1379.232731   
       2007-01-09 19:00:00  1378

## Multipliers and tick size

Each contract may have different multipliers. That means, that a 1$ move is equal to a 50$ change in returns for each contract one is exposed to. We will create a dictionary with the relevant information to calculate the returns down the road.

### Compute Functions

In [ ]:
def calcular_retorno(simbolo, precio_anterior, precio_actual):
    """Compute returns. takes symbol, beginning price, end price. Returns float"""
    params = instrumentos[simbolo]
    delta_precio = precio_actual - precio_anterior
    retorno = delta_precio * params['multiplicador']
    return retorno

: 

In [ ]:

# Simulate buy and hold from the beginning of the data
def simular_buy_and_hold(history_df, simbolo):
    precio_inicial = history_df['close'].iloc[0]
    precio_final = history_df['close'].iloc[-1]
    retorno_total = calcular_retorno(simbolo, precio_inicial, precio_final)
    return retorno_total

: 

In [21]:
# Definir la función para calcular la fecha de expiración
def calcular_fecha_expiracion(anio, mes):
    primer_dia_mes = datetime(anio, mes, 1)
    primer_viernes = primer_dia_mes + timedelta(days=(4 - primer_dia_mes.weekday() + 7) % 7)
    tercer_viernes = primer_viernes + timedelta(weeks=2)
    return tercer_viernes

# Crear un diccionario con las fechas de expiración para ES y MES
fechas_expiracion = {
    'ES': [calcular_fecha_expiracion(anio, mes) for anio in range(2007, 2024) for mes in [3, 6, 9, 12]],
    'MES': [calcular_fecha_expiracion(anio, mes) for anio in range(2019, 2024) for mes in [3, 6, 9, 12]]
}

# Verificar el contenido del diccionario de fechas de expiración
print(fechas_expiracion)



{'ES': [datetime.datetime(2007, 3, 16, 0, 0), datetime.datetime(2007, 6, 15, 0, 0), datetime.datetime(2007, 9, 21, 0, 0), datetime.datetime(2007, 12, 21, 0, 0), datetime.datetime(2008, 3, 21, 0, 0), datetime.datetime(2008, 6, 20, 0, 0), datetime.datetime(2008, 9, 19, 0, 0), datetime.datetime(2008, 12, 19, 0, 0), datetime.datetime(2009, 3, 20, 0, 0), datetime.datetime(2009, 6, 19, 0, 0), datetime.datetime(2009, 9, 18, 0, 0), datetime.datetime(2009, 12, 18, 0, 0), datetime.datetime(2010, 3, 19, 0, 0), datetime.datetime(2010, 6, 18, 0, 0), datetime.datetime(2010, 9, 17, 0, 0), datetime.datetime(2010, 12, 17, 0, 0), datetime.datetime(2011, 3, 18, 0, 0), datetime.datetime(2011, 6, 17, 0, 0), datetime.datetime(2011, 9, 16, 0, 0), datetime.datetime(2011, 12, 16, 0, 0), datetime.datetime(2012, 3, 16, 0, 0), datetime.datetime(2012, 6, 15, 0, 0), datetime.datetime(2012, 9, 21, 0, 0), datetime.datetime(2012, 12, 21, 0, 0), datetime.datetime(2013, 3, 15, 0, 0), datetime.datetime(2013, 6, 21, 0, 0)

In [13]:
instrumentos = {
    'ES': {
        'multiplicador': 50,
        'valor_tick': 0.25,
        'fluctuacion_minima': 12.50
    },
    'MES': {
        'multiplicador': 5,
        'valor_tick': 0.25,
        'fluctuacion_minima': 1.25
    }
}


In [11]:
retorno_es = simular_buy_and_hold(es_history, 'ES')
retorno_mes = simular_buy_and_hold(mes_history, 'MES')

print(f"Retorno total ES: {retorno_es}")
print(f"Retorno total MES: {retorno_mes}")


Retorno total ES: 197502.44385132618
Retorno total MES: 11431.04044389167
